In [1]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
import os

In [2]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader("data", use_multithreading=True, show_progress=True)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

  0%|          | 0/1675 [00:00<?, ?it/s]/opt/homebrew/anaconda3/envs/OpenAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 1/1675 [00:04<2:08:28,  4.60s/it][WARNING] Could not load include file <isonum.txt> at data/wpilib_docs/beta/tasks/task-0-tools-and-docs.rst line 3 column 1
[WARNING] Reference not found for 'Key "reg"' at data/wpilib_docs/beta/tasks/task-0-tools-and-docs.rst line 6 column 73

  0%|          | 2/1675 [00:05<1:11:58,  2.58s/it][WARNING] Could not load include file <isonum.txt> at data/wpilib_docs/beta/tasks/task-1-software-install.rst line 3 column 1
[WARNING] Reference not found for 'Key "reg"' at data/wpilib_docs/beta/tasks/task-1-software-install.rst line 6 column 83

  0%|          | 4/1675 [00:07<36:59,  1.33s/it]  [WARNING] Could not load include file <isonum.

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

index_name = "langchain-demo"

# First, check if our index already exists. If it doesn't, we create it
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)